In [1]:
import pandas as pd
import numpy as np
import os, json
from core import start_calc, start

df = pd.read_csv("../data/CPED/valid_split.csv", usecols=["Speaker", "Utterance",
                                                          "Neuroticism", "Extraversion", "Openness",
                                                          "Agreeableness", "Conscientiousness"])
df = df[df["Agreeableness"] != "unknown"] # 剔除没有人格的人物
df.head(1)

,Speaker,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness,Utterance
0,于春晓,high,high,low,high,high,怎么了


In [2]:
# 真实值
r_real = pd.DataFrame([], columns=["Speaker",
                                   "Neuroticism", "Extraversion", "Openness",
                                   "Agreeableness", "Conscientiousness"])
for v, row in pd.DataFrame(df.groupby(["Speaker"])["Utterance"].count()).sort_values(by=["Utterance"], ascending=True).iloc[:, :].iterrows():
    if not np.any(r_real["Speaker"] == v):
        # 保存真实值
        tmpB = df[df["Speaker"] == v].iloc[1, :]
        r_real = pd.concat([r_real, pd.DataFrame(
            [df[df["Speaker"] == v].iloc[1, :-1]]
        )])
r_real.head(3)

,Speaker,Neuroticism,Extraversion,Openness,Agreeableness,Conscientiousness
6946,安娜,high,high,low,high,high
6237,袁大头,low,high,high,high,low
2571,胡七星,low,high,high,high,high


In [3]:
log_speaker = []
log_send = []
log_content = []
for v, row in pd.DataFrame(
    df.groupby(["Speaker"])["Utterance"].count()).sort_values(by=["Utterance"],
                                                              ascending=True).iloc[:, :].iterrows():
    use_sentence = ""
    use_token = 0
    for i, row_j in df[df["Speaker"] == v].iterrows():
        use_token_sub = start_calc(use_sentence + row_j["Utterance"])
        if use_token_sub > 3500:  # 如果新增的超出token了，清空一下
            log_speaker.append(v)
            log_send.append(use_sentence)
            log_content.append(start(use_sentence))  # 咨询gpt
            use_sentence = "{}: {}".format(v, row_j["Utterance"])  # 重置句子
        else:
            use_sentence += "\n{}: {}".format(v, row_j["Utterance"])
    log_speaker.append(v)
    log_send.append(use_sentence)
    log_content.append(start(use_sentence))  # 咨询gpt


In [4]:
pd.DataFrame(
    np.array([[log_speaker[i], log_send[i], log_content[i]] for i, v in enumerate(log_speaker)], dtype=np.str0),
    columns=["Speaker", "Utter", "Res_Utter"]).to_csv("../data/self/gpt3.5_5.csv", index = False)

上面获取了gpt的结果，并进行了保存，下面就开始解析

In [70]:
import re
# 预测值
r_pre = pd.DataFrame([], columns=["Speaker",
                                  "Neuroticism", "Extraversion", "Openness",
                                  "Agreeableness", "Conscientiousness"])
gpt_res = pd.read_csv("../data/self/gpt4_1.csv")
a = []
for v, row in gpt_res.iterrows():
    res_arr = re.findall(r"\{.*?\}", row["Res_Utter"], re.S)
    try:
        arr = json.loads(res_arr[0])
        r_pre = pd.concat([r_pre, pd.DataFrame([
            [row["Speaker"], arr["神经质"], arr["外向性"],
                arr["开放性"], arr["宜人性"], arr["尽责性"]]],
            columns=["Speaker",
                     "Neuroticism", "Extraversion", "Openness",
                     "Agreeableness", "Conscientiousness"])])
    except:
        continue

In [71]:
# 转换为 0 0.5 1
y_real = r_real
y_pre = r_pre
for v in ["Neuroticism", "Extraversion", "Openness","Agreeableness", "Conscientiousness"]:
    y_pre[v] = y_pre[v].replace({"low": 0, "high": 1, "medium": 0.5})
    y_real[v] = y_real[v].replace({"low": 0, "high": 1, "medium": 0.5})

y_pre = y_pre.groupby(by=["Speaker"]).mean().applymap(lambda x: 1 if x > 0 else 0)
y_real = y_real.groupby(by=["Speaker"]).mean()

# 让真实值和预测值之间的长度统一
y_real = y_real.loc[[np.any(y_pre.index == i) for i in y_real.index],:]

In [72]:
for v in ["Neuroticism", "Extraversion", "Openness", "Agreeableness", "Conscientiousness"]:
    print(v, end="\t")
    print(1 - np.sum(
        [abs(y_real[v][i] - y_pre[v][i]) for i in y_pre.index]) / y_pre.shape[0])

Neuroticism	0.5416666666666667
Extraversion	0.45833333333333337
Openness	0.625
Agreeableness	0.5
Conscientiousness	0.7916666666666666
